<a href="https://colab.research.google.com/github/sunilbabu07/Mcq/blob/main/MCQ_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import random

In [2]:
!pip install PyPDF2 --upgrade
import requests
import PyPDF2

In [3]:
# URL
file_url = 'https://github.com/akaiketech/internship-assignment-nlp/raw/main/Dataset/chapter-2.pdf'

# Download the file
response = requests.get(file_url)

# Save the file to your local system
with open('chapter-2.pdf', 'wb') as pdf_file:
    pdf_file.write(response.content)

# Open the PDF file
with open('chapter-2.pdf', 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Extract text from each page
    context = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        context += page.extract_text()

# Close the PDF file
pdf_file.close()

# Print or manipulate the extracted text as needed
print(context)

From Trade to Territory                   
The Company Establishes Power2
Aurangzeb was the last of the powerful Mughal rulers. He 
established control over a very large part of the territory that is now known as India. After his death in 1707, many Mughal governors ( subadars) and big zamindars 
began asserting their authority and establishing regional kingdoms. As powerful regional kingdoms emerged in various parts of India, Delhi could no longer function as an effective centre.
By the second half of the eighteenth century, 
however, a new power was emerging on the political horizon – the British. Did you know that the British originally came as a small trading company and were reluctant to acquire territories? How then did they come to be masters of a vast empire? In this chapter you will see how this came about.
Fig. 1 – Bahadur Shah Zafar  and his sons being arrested by Captain Hodson
After Aurangzeb there was no 
powerful Mughal ruler, but Mughal emperors continued to be symbolic

In [4]:
# Load the English language model from Spacy
nlp = spacy.load("en_core_web_sm")

In [26]:
# Define a function to generate multiple-choice questions
def generate_mcq(context, num_questions):
    # Process the input context
    doc = nlp(context)

    # Create a set to store unique questions
    unique_questions = set()

    # Define a function to generate a single MCQ with multiple correct answers
    def generate_single_mcq():
        while True:
            # Randomly select a sentence and a word from the context
            sentence = random.choice(list(doc.sents))
            blank_word = random.choice([token for token in sentence if not token.is_punct])
            # Generate a question with a blank
            question_text = sentence.text.replace(blank_word.text, "______")
            # Set the correct answer
            correct_answers = [blank_word.text]

            # Generate other possible answers
            other_options = [token.text for token in doc if token.is_alpha and token.text != correct_answers[0]]

            # Randomly determine correct options
            num_correct_options = 2
            correct_answers.extend(random.sample(other_options, num_correct_options))

            # Determine the number of incorrect options
            num_other_options = min(4 - num_correct_options, len(other_options))
            other_options = random.sample(other_options, num_other_options)

            # Combine everything into an MCQ object
            options = correct_answers + other_options
            random.shuffle(options)
            mcq = {
                "question": question_text,
                "options": options,
                "correct_answers": correct_answers
            }

            # Convert lists to tuples within the MCQ dictionary
            mcq['options'] = tuple(mcq['options'])
            mcq['correct_answers'] = tuple(mcq['correct_answers'])

            # Convert the MCQ dictionary to a tuple
            mcq_tuple = tuple(sorted(mcq.items()))

            # Check if this question is unique
            if mcq_tuple not in unique_questions:
                unique_questions.add(mcq_tuple)
                return mcq

    # Generate a list of MCQs
    mcqs = [generate_single_mcq() for _ in range(num_questions)]

    return mcqs

In [23]:
def create_distractors(word, num_distractors):
    word = nlp(word)
    word_vector = word.vector

    # Find words that are similar to the given word using cosine similarity
    similar_words = [w.text for w in nlp.vocab if w.has_vector and w.is_lower and w.text != word.text and word.similarity(w) > 0.5]

    # Ensure that there are always num_distractors options
    num_available_distractors = min(len(similar_words), num_distractors)

    # Randomly select num_distractors similar words as distractors
    distractors = random.sample(similar_words, num_available_distractors)

    return distractors


In [27]:
# Get user input for the context and the number of questions
num_questions = int(input("Enter the number of questions to generate: "))

# Generate and display the MCQs
mcqs = generate_mcq(context, num_questions)
for i, mcq in enumerate(mcqs, start=1):
    question_text = f"Q{i}: {mcq['question']}"
    options = "\n".join([f"{chr(97+j)}. {option}" for j, option in enumerate(mcq['options'])])
    correct_options = " & ".join([f"({chr(97+mcq['options'].index(ans))})" for ans in mcq['correct_answers']])
    print(f"{question_text}\n{options}\nCorrect Options: {correct_options}\n")



Enter the number of questions to generate: 2
Q1: This was the base from ______ the Company’s traders, known at that time as “factors”, operated.
a. was
b. primarily
c. did
d. which
e. ruler
Correct Options: (d) & (a) & (b)

Q2: But it ______ meant something else.
a. committed
b. of
c. also
d. and
e. not
Correct Options: (c) & (e) & (d)

